In [69]:
import nltk
from nltk import bigrams,trigrams 
from nltk.corpus import reuters
from collections import Counter, defaultdict
from gensim.test.utils import datapath
from gensim.corpora import WikiCorpus as wkc


In [70]:
wikidspath = datapath('enwiki-latest-pages-articles1.xml-p000000010p000030302-shortened.bz2')
sentences = wkc(wikidspath).get_texts()

In [71]:
nltk.download('punkt')
nltk.download('reuters')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package reuters to /root/nltk_data...
[nltk_data]   Package reuters is already up-to-date!


True

In [80]:
reutrsSentence  = reuters.sents()

In [81]:
print(reutrsSentence)

[['ASIAN', 'EXPORTERS', 'FEAR', 'DAMAGE', 'FROM', 'U', '.', 'S', '.-', 'JAPAN', 'RIFT', 'Mounting', 'trade', 'friction', 'between', 'the', 'U', '.', 'S', '.', 'And', 'Japan', 'has', 'raised', 'fears', 'among', 'many', 'of', 'Asia', "'", 's', 'exporting', 'nations', 'that', 'the', 'row', 'could', 'inflict', 'far', '-', 'reaching', 'economic', 'damage', ',', 'businessmen', 'and', 'officials', 'said', '.'], ['They', 'told', 'Reuter', 'correspondents', 'in', 'Asian', 'capitals', 'a', 'U', '.', 'S', '.', 'Move', 'against', 'Japan', 'might', 'boost', 'protectionist', 'sentiment', 'in', 'the', 'U', '.', 'S', '.', 'And', 'lead', 'to', 'curbs', 'on', 'American', 'imports', 'of', 'their', 'products', '.'], ...]


In [74]:
def wordsProbability(wordModel):
  for nextWord in wordModel:
    nextWords = wordModel[nextWord]
    totalWordCount = float(sum(nextWords.values()))
    for previousWord in nextWords:
      nextWords[previousWord] /= totalWordCount

def singleWordProbability(sentenceModel,wordCount):
  for word in sentenceModel:
    sentenceModel[word] /= wordCount

def toLowerCase(s):
  if type(s)==str:
    return s.lower()
  return s

In [106]:
model1 = defaultdict(lambda: set())
model2 = defaultdict(lambda: set())

def wordCount(model3,model4,model5,sentences):
  wordCount = 0
  for sentence in sentences:
    for word in sentence:
      wordCount += 1
      model3[word] += 1
    for previousWord2,previousWord1,nextWord in trigrams(sentence,pad_right=True,pad_left=True):
      previousWord1 = toLowerCase(previousWord1)
      previousWord2 = toLowerCase(previousWord2)
      nextWord = convertToLower(nextWord)
      model4[nextWord][previousWord1] += 1 
      model5[nextWord][previousWord2] += 1
      model1[previousWord1].add(nextWord)
      model2[previousWord2].add(nextWord) 

  return wordCount

In [115]:
model3 = defaultdict(lambda:0)
model4 = defaultdict(lambda: defaultdict(lambda:0))
model5 = defaultdict(lambda: defaultdict(lambda:0))

In [116]:
wikiWordCount = wordCount(model3,model4,model5,sentences)
print(wikiWordCount)

0


In [117]:
reutrsWordCount = wordCount(model3,model4,model5,reutrsSentence)
print(reutrsSentence)

[['ASIAN', 'EXPORTERS', 'FEAR', 'DAMAGE', 'FROM', 'U', '.', 'S', '.-', 'JAPAN', 'RIFT', 'Mounting', 'trade', 'friction', 'between', 'the', 'U', '.', 'S', '.', 'And', 'Japan', 'has', 'raised', 'fears', 'among', 'many', 'of', 'Asia', "'", 's', 'exporting', 'nations', 'that', 'the', 'row', 'could', 'inflict', 'far', '-', 'reaching', 'economic', 'damage', ',', 'businessmen', 'and', 'officials', 'said', '.'], ['They', 'told', 'Reuter', 'correspondents', 'in', 'Asian', 'capitals', 'a', 'U', '.', 'S', '.', 'Move', 'against', 'Japan', 'might', 'boost', 'protectionist', 'sentiment', 'in', 'the', 'U', '.', 'S', '.', 'And', 'lead', 'to', 'curbs', 'on', 'American', 'imports', 'of', 'their', 'products', '.'], ...]


In [118]:
wordsProbability(model4)
wordsProbability(model5)

totalWord = wikiWordCount + reutrsWordCount
print(totalWord)

1720917


In [119]:
singleWordProbability(model3,totalWord)

In [120]:
probability = []
def wordSuggestion(word2,word1): 
  for nextWord in model1[word1] & model2[word2]:
    naiveBayesTrigramValue = model3[nextWord]*model4[nextWord][word1]*model5[nextWord][word2]
    probability.append((nextWord,naiveBayesTrigramValue))

In [114]:
text = input("Text: ")
probability = [] 
text = text.split(" ")
wordSuggestion(text[0],text[1])
ProbabilityWordsList.sort(key=lambda o:o[1],reverse=True)
print(*ProbabilityWordsList[:10])

Text: I have
('to', 1.559359364961502e-08) ('that', 7.821940408821843e-09) ('and', 3.094102899178931e-09) ('it', 2.8855892064629994e-09) ('the', 2.230569445442022e-09) ('in', 1.986452993509535e-09) ('this', 1.306849467794441e-09) ('more', 1.0849316336406677e-09) ('of', 8.868631958384381e-10) ('little', 0.0)
